In [32]:
# !pip install beautifulsoup4
# !pip install selenium
# !pip install webdriver_manager

In [12]:
import requests
#from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
# from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select

In [13]:
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())
URL = "https://www.allmusic.com/advanced-search?1666087655301"
driver.get(URL)

[WDM] - Downloading: 100%|██████████| 6.29M/6.29M [00:01<00:00, 5.26MB/s]
C:\Users\Nastya\AppData\Local\Temp\ipykernel_16924\2469949736.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())


In [14]:
all_genres = [('rap', '//*[@id="MA0000002816"]'), ('folk', '//*[@id="MA0000002592"]'), ('hard rock', '//*[@id="MA0000002636"]'),
             ('disco', '//*[@id="MA0000002552"]'), ('electronic', '//*[@id="MA0000002572"]')]

In [18]:
def get_img_urls(genres, limit):
    
    img_urls = [[], [], [], [], []]
    
    select = Select(driver.find_element(By.CLASS_NAME, 'start-year'))
    select.select_by_value('2000')
    
    input_filter_genres = driver.find_element(By.XPATH, "//input[@placeholder='type to filter genres & styles by name']")
    
    clear_input_filter_genres = driver.find_element(By.CLASS_NAME, 'clear')

    i = 0
    for genre in genres:
        
        input_filter_genres.send_keys(genre[0])
        checkbox = driver.find_element(By.XPATH, genre[1])
        actions = ActionChains(driver)
        actions.move_to_element(checkbox).perform()
        checkbox.click()
        driver.implicitly_wait(5)
        
        while (len(img_urls[i]) < limit):
            
            tbody = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "tbody")))
#             tbody = driver.find_element(By.TAG_NAME, "tbody")
            tbody_elems = tbody.find_elements(By.TAG_NAME, "img")
            for elem in tbody_elems:
                 img_urls[i].append(elem.get_attribute('src'))
                    
            try:
                next_page = driver.find_element(By.XPATH, '//a[contains(text(), "Next »")]')
            except:
                print('Scrapped ' + str(len(img_urls[i])) + ' urls for ' + genre[0] +'. No more available pages.')
                break
            else:
                driver.implicitly_wait(5)
                actions = ActionChains(driver)
                actions.move_to_element(next_page).perform()
                try:
                    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, 
                                                                            '//a[contains(text(), "Next »")]'))).click()
                except:
                    actions.move_to_element(next_page).perform()
                    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, 
                                                                            '//a[contains(text(), "Next »")]'))).click()                    

        print('Scrapped ' + str(len(img_urls[i])) + ' urls for ' + genre[0] + '.')
        i += 1
        checkbox.click()
        clear_input_filter_genres.click()
        
    return img_urls

In [21]:
urls = get_img_urls(all_genres, 2000)

Scrapped 2039 urls for rap.
Scrapped 2039 urls for folk.
Scrapped 2038 urls for hard rock.
Scrapped 2039 urls for disco.
Scrapped 2000 urls for electronic.


In [22]:
driver.quit()

In [44]:
len(urls[1])

2039

In [47]:
def save_imgs(urls, g, genre):
    i = 0
    empty_imgs = 0
    for url in urls[g]:
        if url == 'https://cdn-gce.allmusic.com/images/no_image/album_300x300.png':
            empty_imgs += 1
        else:
            img_data = requests.get(url).content
            with open(path_to_images_folder + genre + '\\image' + str(i) + '.jpg', 'wb') as handler:
                handler.write(img_data)
            i += 1
    print(str(empty_imgs) + ' empty images found')

In [49]:
save_imgs(urls, 0, 'rap')

105 empty images found


In [41]:
e_i = save_imgs(urls, 1, 'folk')

In [43]:
print(str(e_i) + ' empty images found')

139 empty images found


In [45]:
save_imgs(urls, 2, 'rock')

127

In [46]:
save_imgs(urls, 3, 'disco')

256

In [48]:
save_imgs(urls, 4, 'electro')

136 empty images found
